Ring Polymer Melt
===========

insert description

### Concepts Used
- blergh

### Tools Used
- blergh

References
-----------
 
  

Global Imports and Setup
----------------------------

In [ ]:
import numpy as np
import holoviews as hv
hv.extension('bokeh')

%load_ext autoreload
%autoreload 2

In [ ]:
%opts Curve Scatter [width=500,height=400] Layout [shared_axes=False] Scatter (size=10,alpha=0.5)
%opts Curve Scatter [fontsize={'xlabel':14,'xlabel':14,'ylabel':14,'ticks':12}]
%opts Overlay [legend_position='bottom_left']
%opts Layout [shared_axes=False]


colors = {}
colors[2000] = 'blue'
colors[500] = 'red'
colors[100] = 'green'

ls = {}
ls[2000] = 'solid'
ls[500] = 'dashed'
ls[100] = 'dotted'

markers = {}
markers[2000] = 'o'
markers[500] = '^'
markers[100] = 'd'

Demo
-----

In [ ]:
%%opts Overlay [legend_position='bottom_right']
from math import sqrt

extents = (0,0,13,1.0)

gr_plots = []
for name,N,x,y in gr_results:
    label = 'N={} (typyPRISM)'.format(N)
    style = {'line_dash':ls[N],'color':colors[N]}
    c1 = hv.Curve((x,y),label=label,extents=extents)(style=style)
    gr_plots.append(c1)
    
for N,x,y in gr_compare:
    label = 'N={} (Ref 1)'.format(N)
    style = {'marker':markers[N],'color':colors[N]}
    c1 = hv.Scatter((x,y),label=label,extents=extents)(style=style)
    gr_plots.append(c1)
    
    
hv.Overlay(gr_plots).redim.label(x='r',y='g(r)')

# KOYAMA 

In [ ]:
# %load ../typyPRISM/omega/Koyama.py
#!python
from __future__ import division,print_function
from typyPRISM.omega.Omega import Omega
import numpy as np
from math import exp,sin,cos,sqrt

class Koyama(Omega):
    '''Semi-flexible Koyama-based intra-molecular correlation function

    .. warning::
        This function is very sensitive to the bending energy parameter
        and will crash if the value is too low (must be>0) or too high.
        The exact upper value also changes with changing chain length.
        
    
    .. note::
        Kevin G. Honnell, John G. Curro, Kenneth S. Schweizer
        Local structure of semiflexible polymer melts
        Macromolecules, 1990, 23 (14), pp 3496–3505
    
    Attributes
    ----------
    sigma: float
        contact distance between these sites (i.e. site diameter)

    l: float
        bond length
        
    length: float
        number of monomers/sites in the chain

    epsilon: float
        bending energy parameter. Must be >0.
        
    lp: float
        persistence length
    '''
    def __init__(self,epsilon,sigma,l,length):
        self.epsilon = epsilon
        self.sigma   = sigma
        self.length  = length
        self.l       = l
        self.cos0    = 1 - sigma*sigma/(2.0 * l * l)
        self.value   = None
        
        self.lp = l/(1+self.cos_avg(epsilon))
    def cos_avg(self,epsilon):
        '''First moment of bond angle distribution'''
        e = epsilon
        cos0 = self.cos0
        return 1/e  - ( exp(e) + cos0*exp(-e*cos0) )/( exp(e) - exp(-e*cos0) )
    
    def cos_sq_avg(self,epsilon):
        '''Second moment of bond angle distribution'''
        e = epsilon
        cos0 = self.cos0
        cos1 = self.cos_avg(epsilon)
        return (2/e)*cos1 + ( exp(e) - cos0*cos0*exp(-e*cos0) )/( exp(e) - exp(-e*cos0) )
    
    def koyama_kernel(self,k,cos1,cos2,n):
        '''
        Apologies for the mess below, but this is essentially how the math is 
        presented in the paper. Please see equation 18 of the above reference 
        and if there is a better, cleaner way to do this, make it so
        '''
        l = self.l
        q = -cos1
        p = (3*cos2 - 1)/2
        
        D  = n * n * ((1 + q)/(1 - q))**(2.0) 
        D -= n*(1 + (2*q/(1-q)**(3.0)) * (6 + 5*q + 3*q*q) - 4*p/(1-p)*((1 + q)/(1 - q))**(2.0))
        D += 2*q/(1-q)**(4.0) * (4 + 11*q + 12*q*q)
        D -= 4*p/(1-p) * (1 + 8*q/(1-q)**(3.0) + p/(1-p)*((1 + q)/(1 - q))**(2.0))
        D -= q**(n) * 8*q/(1-q)**(3.0) * (n*(1 + 3*q))
        D -= q**(n) * 8*q/(1-q)**(3.0) * ((1 + 2*q + 3*q*q)/(1-q))
        D -= q**(n) * 8*q/(1-q)**(3.0) * (-2*p/(q-p)**(2.0) *(n*(1-q)*(q-p)+2*q*q-q*p-p))
        D -= 6*q**(2*n+2)/(1-q)**(4.0)
        D += p**(n) * (4/(1-p) * (1 + 8*q/(1-q)**(3.0) - ((1+q)/(1-q))**2.0 * (1 - p/(1-p)) ))
        D -= p**(n) * (16*q*q/(1-q)**(3.0) * (1/(q-p)**(2.0))*(q+q*q-2*p))
        D *= 2/3
        
        r2 = n*l*l*((1-cos1)/(1+cos1) + 2*cos1/n * (1-(-cos1)**(n))/(1 + cos1)**(2.0))
        r4 = r2*r2 + l*l*l*l*D
        
        try:
            C = sqrt(0.5 * (5 - 3*r4/(r2*r2)))
            B = sqrt(C*r2)
            A = sqrt(r2*(1-C)/6)
        except ValueError as e:
            raise ValueError('Bad chain parameters. (Try reducing epsilon)')
            
        return np.sin(B*k)/(B*k) * np.exp(-A*A*k*k)

    
    def __repr__(self):
        return '<Omega: Koyama>'
    
    def calculate(self,k):
        self.value = np.zeros_like(k)
        
        cos1 = self.cos_avg(self.epsilon)
        cos2 = self.cos_sq_avg(self.epsilon)
        for i in range(1,self.length-1):
            for j in range(i+1,self.length):
                n = abs(i - j)
                self.value += self.koyama_kernel(k=k,cos1=cos1,cos2=cos2,n=n)
        self.value *= 2/self.length
        self.value += 1.0
        
        return self.value
        
        

sigma = 1.0
lb = 1.0
N = 200
eps = 0.15

K = Koyama(sigma=sigma,l=lb,length=N,epsilon=eps)
d = Domain(dr = 0.1,length=1024) 

K.cos_avg(eps)
K.lp

lp = 1.38
cos1 = 1 - l/lp


# O = K.calculate(d.k)
# siglj  = 1/(1.0148)
# x = siglj * d.k
# y = siglj * siglj * d.k * d.k * O
# hv.Curve((x,y),extents=(0,0,4,10)).redim.label(x='k',y='k*k*O')

In [ ]:
from math import sqrt,sin,exp




f1 = lambda e: 1/e         - ( np.exp(e) + cos0*np.exp(-e*cos0) )/( np.exp(e) - np.exp(-e*cos0) )
f2 = lambda e: (2/e)*f1(e) + ( np.exp(e) - cos0*cos0*np.exp(-e*cos0) )/( np.exp(e) - np.exp(-e*cos0) )
def koyama_omega(k,i,j,e,lb=1.0,sigma=1.0):
    cos0 = 1 - sigma*sigma/(2.0 * lb * lb)
    cos  = f1(e)
    cos2 = f2(e)
    lp   = lb/(1+cos)
    q    = -cos
    p    = (3*cos2 - 1)/2
    n    = abs(i - j)
    
    # Apologies for the mess below. Please see equation 18 of the above reference and if 
    # there is a better, cleaner way to do this, make it so
    D  = n * n * ((1 + q)/(1 - q))**(2.0) 
    D -= n*(1 + (2*q/(1-q)**(3.0)) * (6 + 5*q + 3*q*q) - 4*p/(1-p)*((1 + q)/(1 - q))**(2.0))
    D += 2*q/(1-q)**(4.0) * (4 + 11*q + 12*q*q)
    D -= 4*p/(1-p) * (1 + 8*q/(1-q)**(3.0) + p/(1-p)*((1 + q)/(1 - q))**(2.0))
    D -= q**(n) * 8*q/(1-q)**(3.0) * (n*(1 + 3*q))
    D -= q**(n) * 8*q/(1-q)**(3.0) * ((1 + 2*q + 3*q*q)/(1-q))
    D -= q**(n) * 8*q/(1-q)**(3.0) * (-2*p/(q-p)**(2.0) *(n*(1-q)*(q-p)+2*q*q-q*p-p))
    D -= 6*q**(2*n+2)/(1-q)**(4.0)
    D += p**(n) * (4/(1-p) * (1 + 8*q/(1-q)**(3.0) - ((1+q)/(1-q))**2.0 * (1 - p/(1-p)) ))
    D -= p**(n) * (16*q*q/(1-q)**(3.0) * (1/(q-p)**(2.0))*(q+q*q-2*p))
    D *= 2/3
    
    r2 = n*lb*lb*((1-cos)/(1+cos) + 2*cos/n * (1-(-cos)**(n))/(1 + cos)**(2.0))
    r4 = r2*r2 + lb*lb*lb*lb*D
    
    try:
        C = sqrt(0.5 * (5 - 3*r4/(r2*r2)))
        B = sqrt(C*r2)
        A = sqrt(r2*(1-C)/6)
    except ValueError as e:
        raise ValueError('Bad chain parameters. (Try reducing epsilon)')
        
    return np.sin(B*k)/(B*k) * np.exp(-A*A*k*k)


from typyPRISM import Domain

sigma = 1.0
lb = 1.0
N = 200
eps = 0.3
eps = 0.15

lp = lb/(1+f1(eps))
print(lp/lb)

d = Domain(dr = 0.1,length=1024)
omega = np.zeros_like(d.k)

for i in range(1,N-1):
    for j in range(i+1,N):
        omega += koyama_omega(k=d.k,i=i,j=j,e=eps,lb=lb,sigma=sigma)
omega *= 2/N        
omega += 1.0
        
        
siglj  = 1/(1.0148)
x = siglj * d.k
y = siglj * siglj * d.k * d.k * omega
hv.Curve((x,y),extents=(0,0,4,10)).redim.label(x='k',y=(k*k*O))
        